# Oanda: How to create a stop loss & take profit

In [26]:
from trader import ConTrader

In [28]:
trader

<module 'trader' from 'C:\\Users\\audri\\Desktop\\AutoTrader\\finiacialDataAnalysis\\StopLoss\\trader.py'>

In [38]:
trader = ConTrader("Oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [44]:
trader.get_most_recent()

In [46]:
trader.define_strategy()

In [52]:
current_price = trader.data[trader.instrument].iloc[-1]
current_price

1.13668

In [56]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                           sl_distance = 0.01)
order

{'id': '351',
 'time': '2025-04-17T03:53:59.546613963Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '351',
 'requestID': '61361677361058542',
 'type': 'MARKET_ORDER',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'timeInForce': 'FOK',
 'positionFill': 'DEFAULT',
 'reason': 'CLIENT_ORDER',
 'stopLossOnFill': {'distance': '0.01', 'timeInForce': 'GTC'}}

In [58]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '354',
 'time': '2025-04-17T03:56:18.626449965Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '353',
 'requestID': '115404873470446073',
 'type': 'ORDER_FILL',
 'orderID': '353',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13648,
 'fullVWAP': 1.13648,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13648, 'liquidity': '500000'},
   {'price': 1.13647, 'liquidity': '2500000'},
   {'price': 1.13646, 'liquidity': '2000000'},
   {'price': 1.13645, 'liquidity': '5000000'},
   {'price': 1.13642, 'liquidity': '10000000'},
   {'price': 1.13639, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13662, 'liquidity': '500000'},
   {'price': 1.13663, 'liquidity': '500000'},
   {'price': 1.13664, 'liquidity': '2000000'},
   {'price': 1.13665, 'liquidity': '2000000'},
   {'price': 1.13666, 'liquidity': '5000000'},
   {'price': 1.13668, 'liquidity': '10000000'},
   {'